In [ ]:
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
!pip install seqeval

In [ ]:
from datasets import load_dataset
datasets = load_dataset("cfilt/HiNER-original")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
label_list = ['B-FESTIVAL', 'B-GAME', 'B-LANGUAGE', 'B-LITERATURE', 'B-LOCATION', 'B-MISC', 'B-NUMEX', 'B-ORGANIZATION', 'B-PERSON', 'B-RELIGION', 'B-TIMEX', 'I-FESTIVAL', 'I-GAME', 'I-LANGUAGE', 'I-LITERATURE', 'I-LOCATION', 'I-MISC', 'I-NUMEX', 'I-ORGANIZATION', 'I-PERSON', 'I-RELIGION', 'I-TIMEX', 'O']
labels_vocab = {label: i for i, label in enumerate(label_list)}
labels_vocab_reverse = {i: label for label, i in labels_vocab.items()}

train_dataset = datasets["train"]
val_dataset = datasets["validation"]
test_dataset = datasets["test"]

In [ ]:
print(len(label_list))

23


In [ ]:
# update the model name
model_name = "xlm-roberta-base"


In [ ]:
# load tokenizer and model
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse
)
label_all_tokens = False


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/75827 [00:00<?, ? examples/s]

Map:   0%|          | 0/10851 [00:00<?, ? examples/s]

Map:   0%|          | 0/21657 [00:00<?, ? examples/s]

In [ ]:
print(train_tokenized[1])

{'id': '1', 'tokens': ['देर', 'रात', 'तक', 'जुहू', 'चौपाटी', 'में', 'यह', 'नजारा', 'आम', 'है', '।'], 'ner_tags': [22, 22, 22, 4, 15, 22, 22, 22, 22, 22, 22], 'input_ids': [0, 105573, 36572, 7231, 35870, 61698, 33807, 174012, 421, 4239, 1618, 10886, 2815, 26046, 460, 207, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 22, 22, 22, 4, -100, 15, -100, 22, 22, 22, -100, -100, 22, 22, 22, -100]}


In [ ]:
for key, value in train_tokenized[1].items():
  print(f"Key: {key}, Value: {value}")



Key: id, Value: 1
Key: tokens, Value: ['देर', 'रात', 'तक', 'जुहू', 'चौपाटी', 'में', 'यह', 'नजारा', 'आम', 'है', '।']
Key: ner_tags, Value: [22, 22, 22, 4, 15, 22, 22, 22, 22, 22, 22]
Key: input_ids, Value: [0, 105573, 36572, 7231, 35870, 61698, 33807, 174012, 421, 4239, 1618, 10886, 2815, 26046, 460, 207, 2]
Key: attention_mask, Value: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Key: labels, Value: [-100, 22, 22, 22, 4, -100, 15, -100, 22, 22, 22, -100, -100, 22, 22, 22, -100]


In [ ]:
# training arguments
from transformers import TrainingArguments, Trainer


In [ ]:
from transformers import TrainingArguments
help(TrainingArguments)


Help on class TrainingArguments in module transformers.training_args:

class TrainingArguments(builtins.object)
 |  TrainingArguments(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.Sch

In [ ]:
args = TrainingArguments(
    "HiNER",
    eval_strategy="steps",
    learning_rate=4e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
    eval_steps=10000,
    save_steps=10000,
)


In [ ]:
from transformers import DataCollatorForTokenClassification
from evaluate import load


data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load("seqeval")


In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:

from huggingface_hub import login
import os

# Login from a secure token (set in environment or secrets)
hf_token = os.getenv("HF_TOKEN")
if hf_token:
    login(token=hf_token)
else:
    print("Warning: No Hugging Face token found in environment.")


trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Train the model
trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=9479, training_loss=0.07658825570589531, metrics={'train_runtime': 1916.8647, 'train_samples_per_second': 39.558, 'train_steps_per_second': 4.945, 'total_flos': 2365028421209868.0, 'train_loss': 0.07658825570589531, 'epoch': 1.0})

In [ ]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.0854271799325943,
 'eval_precision': 0.8939193959298934,
 'eval_recall': 0.9062281206727553,
 'eval_f1': 0.9000316771098115,
 'eval_accuracy': 0.9732306920376904,
 'eval_runtime': 100.9067,
 'eval_samples_per_second': 214.624,
 'eval_steps_per_second': 26.837,
 'epoch': 1.0}

In [ ]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)

{'FESTIVAL': {'precision': np.float64(0.5555555555555556), 'recall': np.float64(0.7692307692307693), 'f1': np.float64(0.6451612903225806), 'number': np.int64(39)}, 'GAME': {'precision': np.float64(0.5838509316770186), 'recall': np.float64(0.7642276422764228), 'f1': np.float64(0.6619718309859156), 'number': np.int64(369)}, 'LANGUAGE': {'precision': np.float64(0.9140495867768595), 'recall': np.float64(0.9294117647058824), 'f1': np.float64(0.9216666666666666), 'number': np.int64(1190)}, 'LITERATURE': {'precision': np.float64(0.6273584905660378), 'recall': np.float64(0.7430167597765364), 'f1': np.float64(0.680306905370844), 'number': np.int64(179)}, 'LOCATION': {'precision': np.float64(0.9588511460922398), 'recall': np.float64(0.9530844479936115), 'f1': np.float64(0.9559591004092463), 'number': np.int64(40072)}, 'MISC': {'precision': np.float64(0.6384096024006002), 'recall': np.float64(0.7990610328638498), 'f1': np.float64(0.7097581317764805), 'number': np.int64(1065)}, 'NUMEX': {'precisio

In [ ]:

# testing the model
from transformers import pipeline
import pandas as pd

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
example = "भारतीय अंतरिक्ष अनुसंधान संगठन या इसरो भारत की राष्ट्रीय अंतरिक्ष एजेंसी है, जिसका मुख्यालय सितारे के शहर मुंबई में है। यह अंतरिक्ष विभाग के तहत संचालित होता है जिसकी देखरेख सीधे भारत के प्रधान मंत्री करते हैं जबकि इसरो के अध्यक्ष डीओएस के कार्यकारी के रूप में भी कार्य करते हैं।"

ner_results = nlp(example)
pd.DataFrame(ner_results)

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/google/colab/_dataframe_summarizer.py:57: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(column, errors="raise")


,entity_group,score,word,start,end
0,ORGANIZATION,0.987004,भारतीय अंतरिक्ष अनुसंधान संगठन,0,30
1,ORGANIZATION,0.938460,इसरो,34,38
2,LOCATION,0.994886,भारत,39,43
3,LOCATION,0.895821,,92,93
4,LOCATION,0.640880,सितारे,92,98
5,LOCATION,0.994769,मुंबई,106,111
6,ORGANIZATION,0.525450,अंतर,123,127
7,LOCATION,0.991891,भारत,179,183
8,ORGANIZATION,0.841173,इसरो,215,219
9,ORGANIZATION,0.873698,डी,231,233
